In [2]:
%load_ext autoreload
%autoreload 2

from backtest.domains.strategy import StockData
from backtest.domains.strategy import Strategy
from backtest.domains.strategy_result import StrategyResult,StrategyResultColumnType
from backtest.domains.backtest import Backtest
from backtest.repository.webrepo.bithumb_repo import BithumbRepo
from backtest.repository.webrepo.upbit_repo import UpbitRepo
from backtest.repository.webrepo.binance_repo import BinanceRepo
from backtest.use_cases.stockdata_from_repo import stockdata_from_repo
from backtest.request.stockdata_from_repo import build_stock_data_from_repo_request
from backtest.use_cases.strategy_execute import rsi_function, rsi_big_stock_function
from backtest.use_cases.backtest_execute import backtest_execute

import pandas as pd

request = build_stock_data_from_repo_request(
    filters={'order__eq': 'DOGE', 'from__eq': '2019-01-01'})
response = stockdata_from_repo(BithumbRepo(), request=request)

request3 = build_stock_data_from_repo_request(
    filters={'order__eq': 'XRP', 'from__eq': '2019-01-01'})
response3 = stockdata_from_repo(BithumbRepo(), request=request)

request2 = build_stock_data_from_repo_request(
    filters={'order__eq': 'BTC', 'from__eq': '2019-01-01'})
response2 = stockdata_from_repo(BithumbRepo(), request=request2)
stockdata = response.value
stockdata3 = response.value
btc_stock = response2.value

strategy1 = Strategy(name='rsi_function', function=rsi_function,weight=1,options={'rolling':100})
strategy = Strategy(name='rsi_function2', function=rsi_big_stock_function,weight=1,options={'big_stock':btc_stock,'rolling':100})
backtest = Backtest(strategy_list=[strategy], stockdata_list=[stockdata,btc_stock,stockdata3])
backtest_result = backtest_execute(backtest).value


In [317]:
backtest_result.value['total_profit'].expanding().sum()

date
2019-01-01      0.000000
2019-01-02      0.000000
2019-01-03      0.000000
2019-01-04      0.000000
2019-01-05      0.000000
                 ...    
2023-02-18    423.137945
2023-02-19    423.137945
2023-02-20    423.137945
2023-02-21    423.137945
2023-02-22    423.137945
Freq: D, Name: total_profit, Length: 1514, dtype: float64

In [318]:
pd.options.display.max_colwidth = 1000
backtest_result.value.tail(20)

,total_profit,stock_bucket
date,,
2023-02-03,0.000000,"[(DOGE, 2023-01-21 00:00:00), (DOGE, 2023-01-22 00:00:00), (DOGE, 2023-01-23 00:00:00), (DOGE, 2023-01-24 00:00:00), (DOGE, 2023-01-25 00:00:00), (DOGE, 2023-01-26 00:00:00), (DOGE, 2023-01-27 00:00:00), (DOGE, 2023-01-28 00:00:00), (DOGE, 2023-01-29 00:00:00), (DOGE, 2023-01-30 00:00:00), (DOGE, 2023-01-31 00:00:00), (DOGE, 2023-02-01 00:00:00), (DOGE, 2023-02-02 00:00:00), (DOGE, 2023-02-03 00:00:00), (BTC, 2023-01-21 00:00:00), (BTC, 2023-01-22 00:00:00), (BTC, 2023-01-23 00:00:00), (BTC, 2023-01-24 00:00:00), (BTC, 2023-01-25 00:00:00), (BTC, 2023-01-26 00:00:00), (BTC, 2023-01-27 00:00:00), (BTC, 2023-01-28 00:00:00), (BTC, 2023-01-29 00:00:00), (BTC, 2023-01-30 00:00:00), (BTC, 2023-01-31 00:00:00), (BTC, 2023-02-01 00:00:00), (BTC, 2023-02-02 00:00:00), (BTC, 2023-02-03 00:00:00)]"
2023-02-04,0.000000,"[(DOGE, 2023-01-21 00:00:00), (DOGE, 2023-01-22 00:00:00), (DOGE, 2023-01-23 00:00:00), (DOGE, 2023-01-24 00:00:00), (DOGE, 2023-01-25 00:00:00), (DOGE, 2023-01-26 00:00:00), (DOGE, 2023-01-27 00:00:00), (DOGE, 2023-01-28 00:00:00), (DOGE, 2023-01-29 00:00:00), (DOGE, 2023-01-30 00:00:00), (DOGE, 2023-01-31 00:00:00), (DOGE, 2023-02-01 00:00:00), (DOGE, 2023-02-02 00:00:00), (DOGE, 2023-02-03 00:00:00), (DOGE, 2023-02-04 00:00:00), (BTC, 2023-01-21 00:00:00), (BTC, 2023-01-22 00:00:00), (BTC, 2023-01-23 00:00:00), (BTC, 2023-01-24 00:00:00), (BTC, 2023-01-25 00:00:00), (BTC, 2023-01-26 00:00:00), (BTC, 2023-01-27 00:00:00), (BTC, 2023-01-28 00:00:00), (BTC, 2023-01-29 00:00:00), (BTC, 2023-01-30 00:00:00), (BTC, 2023-01-31 00:00:00), (BTC, 2023-02-01 00:00:00), (BTC, 2023-02-02 00:00:00), (BTC, 2023-02-03 00:00:00), (BTC, 2023-02-04 00:00:00)]"
2023-02-05,0.000000,"[(DOGE, 2023-01-21 00:00:00), (DOGE, 2023-01-22 00:00:00), (DOGE, 2023-01-23 00:00:00), (DOGE, 2023-01-24 00:00:00), (DOGE, 2023-01-25 00:00:00), (DOGE, 2023-01-26 00:00:00), (DOGE, 2023-01-27 00:00:00), (DOGE, 2023-01-28 00:00:00), (DOGE, 2023-01-29 00:00:00), (DOGE, 2023-01-30 00:00:00), (DOGE, 2023-01-31 00:00:00), (DOGE, 2023-02-01 00:00:00), (DOGE, 2023-02-02 00:00:00), (DOGE, 2023-02-03 00:00:00), (DOGE, 2023-02-04 00:00:00), (DOGE, 2023-02-05 00:00:00), (BTC, 2023-01-21 00:00:00), (BTC, 2023-01-22 00:00:00), (BTC, 2023-01-23 00:00:00), (BTC, 2023-01-24 00:00:00), (BTC, 2023-01-25 00:00:00), (BTC, 2023-01-26 00:00:00), (BTC, 2023-01-27 00:00:00), (BTC, 2023-01-28 00:00:00), (BTC, 2023-01-29 00:00:00), (BTC, 2023-01-30 00:00:00), (BTC, 2023-01-31 00:00:00), (BTC, 2023-02-01 00:00:00), (BTC, 2023-02-02 00:00:00), (BTC, 2023-02-03 00:00:00), (BTC, 2023-02-04 00:00:00), (BTC, 2023-02-05 00:00:00)]"
2023-02-06,0.662983,[]
2023-02-07,0.000000,"[(DOGE, 2023-02-07 00:00:00), (BTC, 2023-02-07 00:00:00)]"
2023-02-08,0.000000,"[(DOGE, 2023-02-07 00:00:00), (DOGE, 2023-02-08 00:00:00), (BTC, 2023-02-07 00:00:00), (BTC, 2023-02-08 00:00:00)]"
2023-02-09,0.000000,"[(DOGE, 2023-02-07 00:00:00), (DOGE, 2023-02-08 00:00:00), (DOGE, 2023-02-09 00:00:00), (BTC, 2023-02-07 00:00:00), (BTC, 2023-02-08 00:00:00), (BTC, 2023-02-09 00:00:00)]"
2023-02-10,-0.304831,[]
2023-02-11,0.000000,[]


In [45]:
request = build_stock_data_from_repo_request(
    filters={'order__eq': 'DOGE', 'from__eq': '2023-02-19','chart_interval__eq':'30m','start_time__eq':'06:00','end_time__eq':'23:30'})
response = stockdata_from_repo(UpbitRepo(), request=request)
response.value.data

 2023-02-19 02:00:00


,open,high,low,close,volume
date,,,,,
2023-02-19,114.0,116.0,113.0,115.0,1.601344e+08
2023-02-20,113.0,114.0,111.0,113.0,2.294322e+08
2023-02-21,112.0,115.0,111.0,111.0,3.097848e+08
2023-02-22,112.0,114.0,108.0,111.0,6.798476e+08


In [48]:
request = build_stock_data_from_repo_request(
    filters={'order__eq': 'DOGE', 'from__eq': '2023-02-19','chart_interval__eq':'24h'})
response = stockdata_from_repo(UpbitRepo(), request=request)
response.value.data

 2022-08-07 00:00:00


,open,high,low,close,volume
date,,,,,
2023-02-19,115.0,116.0,112.0,112.0,3.284494e+08
2023-02-20,112.0,114.0,111.0,113.0,3.552030e+08
2023-02-21,113.0,115.0,110.0,111.0,4.544601e+08
2023-02-22,111.0,114.0,108.0,110.0,7.417944e+08


In [34]:

df


            value
2022-01-01     55


,value
2022-01-01 06:00:00,1
2022-01-01 06:30:00,2
2022-01-01 07:00:00,3
2022-01-01 07:30:00,4
2022-01-01 08:00:00,5
2022-01-01 08:30:00,6
2022-01-01 09:00:00,7
2022-01-01 09:30:00,8
2022-01-01 10:00:00,9
2022-01-01 10:30:00,10
